In [10]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = url_input + '?page=' + str(pagenumber)
r = requests.get(url)
bs = BeautifulSoup(r.text, 'html.parser')

In [213]:
url_input =  "http://www.allocine.fr/film/fichefilm-22092/critiques/spectateurs"
pagenumber = 1

url = "{}/?page={}".format(url_input, pagenumber + 1)

In [276]:
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [436]:
def format_comment(comment):
    output_text = ""
    for content in comment.contents:        
        content_str = str(content)        
        content_soup = BeautifulSoup(content_str, 'html.parser')
        spoiler = content_soup.find("span", {"class": "spoiler-content"})
        if spoiler:
            output_text += spoiler.text.strip()
        else:
            output_text += content_str.strip()
    return output_text #.replace('\n', ' ')

def parse_page(page_url):    
    ratings, comments = [], []    
    r = requests.get(page_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # Favorise well structured comments
    # Also avoid sarcastic comments / against main opinion (but it's not a something we need)
    #for helpful in soup.find_all("div", {"class": "reviews-users-comment-useful js-useful-reviews"}):
    #    helpful_dic = json.loads(helpful.get("data-statistics"))
    #    helpful_delta = int(helpful_dic['helpfulCount']) - int(helpful_dic['unhelpfulCount'])
    #    helpfuls.append(helpful_delta)


    for rating in soup.find_all("span", {"class": "stareval-note"}):
        rating_str = str(rating.contents)[2:5]
        rating_float = float(rating_str.replace(',', '.'))
        ratings.append(rating_float)

    for comment in soup.find_all("div", attrs={"class": "content-txt review-card-content"}):        
        comment_text = format_comment(comment)           
        comments.append(comment_text)    
        
    return ratings, comments

In [536]:
import requests
from bs4 import BeautifulSoup

def parse_film(film_url):
    r = requests.get(film_url)
    if r.status_code == requests.codes.not_found:
        # if url is not foud : film does not exist
        return None
    elif len(r.history) > 1: 
        # if there is more than one element in history, the request was redirected
        # and that means there are no "critiques/spectateurs" page      
        return None

    soup = BeautifulSoup(r.text, 'html.parser')
    
    #print("> Film url: " + film_url)
    
    # Find number of pages
    pagination = soup.find("div", {"class": "pagination-item-holder"})
    if pagination:
        #print(pagination)
        pages = pagination.find_all("span")
        #print([page.text for page in pages])
        page_number = int([page.text for page in pages][-1])
    else:
        page_number = 1
    #print("  pages: " + str(page_number))
    
    # Iterate over pages
    film_reviews = [] #defaultdict(tuple)

    for page_id in range(0, page_number):            
        page_url = url = "{film_url}/?page={page_num}".format(
            film_url=film_url,
            page_num=page_id+1)
        ratings, comments = parse_page(page_url)
        
        for rating, comment in zip(ratings, comments):
            film_reviews.append((rating, comment))
            #film_ratings[rating].append(comment)
            
    return film_reviews
            
    # TODO => only keep x comments (random ?)

In [537]:
from collections import defaultdict

def parse_allocine(root_url, max_film_id):   
    #FILM_ID = 22092
    reviews = dict() #defaultdict(list)
    for film_id in range(0, max_film_id+1):
        
        if film_id % 100 == 0:
            print("Progress: {}/{}".format(film_id, max_film_id))
            
        film_url = "{root}/film/fichefilm-{film_id}/critiques/spectateurs".format(
            root=root_url,
            film_id=film_id
        )
        film_reviews = parse_film(film_url)
        if film_reviews:
            reviews[film_id] = film_reviews
    
    return reviews

In [539]:
ROOT_URL = "http://www.allocine.fr"
MAX_FILM_ID = 300
#MAX_REVIEWS_PER_FILM = 30 + keepp only short answers ??

allocine_reviews = parse_allocine(ROOT_URL, MAX_FILM_ID)

Progress: 0/300
Progress: 100/300
Progress: 200/300
Progress: 300/300


In [552]:
film_reviews = allocine_reviews[251]

print("Reviews number : " + str(len(film_reviews)))
for rating, comment in film_reviews:
    print("Rating: {}/5".format(rating))
    print(comment)
    print()

Reviews number : 15
Rating: 3.0/5
Il y a indéniablement une réelle fascination à voir le cinéma rencontrer la peinture, à voir la caméra de Clouzot filmer un tableau de Picasso en train de se faire. Le parti pris du cinéaste est de ne pas montrer le peintre mais (presque) uniquement le tableau et aussi de ne pas filmer le processus créatif en temps réel (un des tableaux projeté en l'espace de cinq minutes a en réalité été peint en cinq heures). L'intérêt de ces choix de mise en scène est de renforcer l'attraction du spectateur pour le peintre car, en ne le voyant pas directement, c'est comme si le tableau se faisait de lui-même. Ce n'est pas Picasso qui corrige ou modifie son oeuvre, mais l'oeuvre elle-même qui change sous les yeux d'un spectateur se devant d'être attentif à tous les mouvements qui défilent sur la toile : Picasso est bien le maître, mais il est invisible; ce ne serait donc pas en le filmant qu'on percerait son mystère et qu'on comprendrait son génie. Pourtant, on peut 

In [550]:
import pickle

PICKLE_PATH = "data/allocine.pickle"

with open(PICKLE_PATH, 'wb') as handle:
    pickle.dump(allocine_reviews, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open(PICKLE_PATH, 'rb') as handle:
#    b = pickle.load(handle)

In [383]:
import json 

pagination = soup.find("div", {"class": "pagination-item-holder"})
pages = pagination.find_all("span")
page_number = [int(page.text) for page in pages][-1]
print(page_number)

for page_index in range(page_number):
    helpfuls = []
    ratings = []
    comments = []
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    

    for helpful in soup.find_all("div", {"class": "reviews-users-comment-useful js-useful-reviews"}):
        helpful_dic = json.loads(helpful.get("data-statistics"))
        helpful_delta = int(helpful_dic['helpfulCount']) - int(helpful_dic['unhelpfulCount'])
        helpfuls.append(helpful_delta)


    for rating in soup.find_all("span", {"class": "stareval-note"}):
        rating_str = str(rating.contents)[2:5]
        rating_float = float(rating_str.replace(',', '.'))
        ratings.append(rating_float)

    for comment in soup.find_all("div", attrs={"class": "content-txt review-card-content"}):
        comment_text = format_comment(comment)
        comments.append(comment_text)
        #print(comment_text)
        #print('--')

    p

#print(pagination)



    #print(comment.contents)
    
    
    
    #spoilers = comment.findAll("span", {"class": "spoiler-content"})
    #print([s.text for s in spoilers])
    #print(comment)
    #text_str = str(comment.text)
    #comments.append(text_str)
    
    
    
"""
for i, (helpful, rating, comment) in enumerate(zip(helpfuls, ratings, comments)):    
    print("Avis : " + str(i+1))
    print("Helpful: {}, Rating: {}".format(helpful, rating))
    print(comment)
    print()   
"""


70
UTF-8


NameError: name 'p' is not defined